# **Project Work Bitcoin Prediction**

Progetto finale per l'accademy Data Science & Generative AI per Crif.

###**Obiettivo di ricerca**

Partendo dal dataset contentente
fluttuazione orarie di diverse
cryptovalute (tra cui Bitcoin),
indici azionari e indici di
sentiment, svolgere un analisi
descrittiva e successivamente un
analisi di Machine Learning per
stimare l’andamento del market
trend di Bitcoin e del fear and
greed index.

### **Fase di ricerca:**
2. Analisi predittiva del market trend di Bitcoin tramite algoritmi ML/NN

## 0. Import

In [ ]:
!pip install kaleido -q
import kaleido #required
kaleido.__version__ #0.2.1

import plotly
plotly.__version__ #5.5.0

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_theme()

In [ ]:
#Tensorflow
import tensorflow
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.metrics import R2Score

import pickle

In [ ]:
#SKLEARN
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor, QuantileRegressor, RANSACRegressor, TheilSenRegressor, LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold, cross_val_score, TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    average_precision_score,
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    confusion_matrix
)

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
#Stats model: per VIF e Modelli Lineari
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_merged = pd.read_csv('/content/drive/MyDrive/Data Science/Machine e Deep Learning/OVED627/Dataset e consegna/merged_fix_to_hour.csv')

data_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17515 entries, 0 to 17514
Columns: 131 entries, Datetime to google_trends_bitcoin
dtypes: float64(128), int64(2), object(1)
memory usage: 17.5+ MB


In [ ]:
columns_BTC = [col for col in data_merged.columns if col.startswith('BTC') or col == 'Datetime']
df = data_merged[columns_BTC]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17515 entries, 0 to 17514
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Datetime            17515 non-null  object 
 1   BTC_USDT_1h_open    17515 non-null  float64
 2   BTC_USDT_1h_high    17515 non-null  float64
 3   BTC_USDT_1h_low     17515 non-null  float64
 4   BTC_USDT_1h_close   17515 non-null  float64
 5   BTC_USDT_1h_volume  17515 non-null  float64
dtypes: float64(5), object(1)
memory usage: 821.1+ KB


## 2. Analsi Predittiva: Market Trend

### 2.1. Preparazione del daset per l'analisi

In [ ]:
columns_to_keep_lstm = []
for col in data_merged.columns:
    col_lower = col.lower()
    if any(key in col_lower for key in ['open', 'high', 'low', 'volume']) and 'close' not in col_lower:
        continue
    columns_to_keep_lstm.append(col)

df_lstm_with_dt = data_merged[columns_to_keep_lstm]
print("Colonne mantenute in df_lstm_with_dt:")
print(df_lstm_with_dt.columns.tolist())

Colonne mantenute in df_lstm_with_dt:
['Datetime', 'BNB_USDT_1h_close', 'BTC_USDT_1h_close', 'DOGE_USDT_1h_close', 'ETH_USDT_1h_close', 'SOL_USDT_1h_close', 'XRP_USDT_1h_close', 'cattle_Close LE=F', 'corn_Close ZC=F', 'crude_Close CL=F', 'gold_Close GC=F', 'silver_Close SI=F', 'soybeans_Close ZS=F', 'wheat_Close ZW=F', 'CAC_Close ^FCHI', 'DAX_Close ^GDAXI', 'Dow_Close ^DJI', 'EURO_Close ^STOXX50E', 'FTSE_Close ^FTSE', 'IBOVESPA_Close ^BVSP', 'IPC_Close ^MXX', 'NASDAQ_Close ^IXIC', 'Russell_Close ^RUT', 'S&P_Close ^GSPC', 'S&P_Close ^GSPTSE', 'VIX_Close ^VIX', 'funding_rate', 'fear_gread_index', 'google_trends_buy_crypto', 'google_trends_bitcoin']


In [ ]:
df_lstm_with_dt['Datetime'] = pd.to_datetime(df_lstm_with_dt['Datetime'])

In [ ]:
df_lstm = df_lstm_with_dt.drop(columns=['Datetime'])
df_lstm.head()

,BNB_USDT_1h_close,BTC_USDT_1h_close,DOGE_USDT_1h_close,ETH_USDT_1h_close,SOL_USDT_1h_close,XRP_USDT_1h_close,cattle_Close LE=F,corn_Close ZC=F,crude_Close CL=F,gold_Close GC=F,...,IPC_Close ^MXX,NASDAQ_Close ^IXIC,Russell_Close ^RUT,S&P_Close ^GSPC,S&P_Close ^GSPTSE,VIX_Close ^VIX,funding_rate,fear_gread_index,google_trends_buy_crypto,google_trends_bitcoin
0,332.3852,30277.44,0.08903,2089.59,24.12,0.5216,164.125,633.25,82.559998,2014.5,...,54699.269531,12119.62793,1792.794678,4139.490234,20583.220703,17.59,0.0001,68.0,20,33
1,332.1377,30240.00,0.08860,2089.27,24.16,0.5195,164.125,633.25,82.559998,2014.5,...,54699.269531,12119.62793,1792.794678,4139.490234,20583.220703,17.59,0.0001,68.0,20,33
2,332.2934,30267.06,0.08896,2088.71,24.47,0.5197,164.125,633.25,82.559998,2014.5,...,54699.269531,12119.62793,1792.794678,4139.490234,20583.220703,17.59,0.0001,68.0,20,33
3,332.2009,30248.96,0.08918,2086.26,24.33,0.5163,164.125,633.25,82.559998,2014.5,...,54699.269531,12119.62793,1792.794678,4139.490234,20583.220703,17.59,0.0001,68.0,20,33
4,333.0055,30302.08,0.09038,2097.00,24.44,0.5193,164.125,633.25,82.559998,2014.5,...,54699.269531,12119.62793,1792.794678,4139.490234,20583.220703,17.59,0.0001,68.0,20,33


In [ ]:
df_lstm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17515 entries, 0 to 17514
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   BNB_USDT_1h_close         17515 non-null  float64
 1   BTC_USDT_1h_close         17515 non-null  float64
 2   DOGE_USDT_1h_close        17515 non-null  float64
 3   ETH_USDT_1h_close         17515 non-null  float64
 4   SOL_USDT_1h_close         17515 non-null  float64
 5   XRP_USDT_1h_close         17515 non-null  float64
 6   cattle_Close LE=F         17515 non-null  float64
 7   corn_Close ZC=F           17515 non-null  float64
 8   crude_Close CL=F          17515 non-null  float64
 9   gold_Close GC=F           17515 non-null  float64
 10  silver_Close SI=F         17515 non-null  float64
 11  soybeans_Close ZS=F       17515 non-null  float64
 12  wheat_Close ZW=F          17515 non-null  float64
 13  CAC_Close ^FCHI           17515 non-null  float64
 14  DAX_Cl

In [ ]:
X_lstm = df_lstm.drop(columns=['BTC_USDT_1h_close'])
#ANALISI VIF
X_lstm_const = sm.add_constant(X_lstm)

# Calcolo VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_lstm.columns
vif_data["VIF"] = [variance_inflation_factor(X_lstm_const.values, i+1) for i in range(len(X_lstm.columns))]  # i+1 per saltare la costante

print(vif_data)

                     feature          VIF
0          BNB_USDT_1h_close    67.221457
1         DOGE_USDT_1h_close    28.263459
2          ETH_USDT_1h_close    27.564141
3          SOL_USDT_1h_close    48.864768
4          XRP_USDT_1h_close    27.883608
5          cattle_Close LE=F     6.871551
6            corn_Close ZC=F    14.438507
7           crude_Close CL=F     5.640558
8            gold_Close GC=F   138.343769
9          silver_Close SI=F    39.915010
10       soybeans_Close ZS=F    22.420619
11          wheat_Close ZW=F     5.014837
12           CAC_Close ^FCHI    64.020848
13          DAX_Close ^GDAXI   229.642738
14            Dow_Close ^DJI   325.367342
15      EURO_Close ^STOXX50E   332.860819
16          FTSE_Close ^FTSE    22.397149
17      IBOVESPA_Close ^BVSP    10.505809
18            IPC_Close ^MXX     9.050063
19        NASDAQ_Close ^IXIC   792.541213
20        Russell_Close ^RUT    33.040408
21           S&P_Close ^GSPC  2223.848204
22         S&P_Close ^GSPTSE   142

In [ ]:
data = {
    "feature": ["BNB_USDT_1h_close", "DOGE_USDT_1h_close", "ETH_USDT_1h_close", "SOL_USDT_1h_close", "XRP_USDT_1h_close",
                "cattle_Close LE=F", "corn_Close ZC=F", "crude_Close CL=F", "gold_Close GC=F", "silver_Close SI=F",
                "soybeans_Close ZS=F", "wheat_Close ZW=F", "CAC_Close ^FCHI", "DAX_Close ^GDAXI", "Dow_Close ^DJI",
                "EURO_Close ^STOXX50E", "FTSE_Close ^FTSE", "IBOVESPA_Close ^BVSP", "IPC_Close ^MXX",
                "NASDAQ_Close ^IXIC", "Russell_Close ^RUT", "S&P_Close ^GSPC", "S&P_Close ^GSPTSE", "VIX_Close ^VIX",
                "funding_rate", "fear_gread_index", "google_trends_buy_crypto", "google_trends_bitcoin"],
    "VIF": [67.22, 28.26, 27.56, 48.86, 27.88, 6.87, 14.44, 5.64, 138.34, 39.91, 22.42, 5.01, 64.02, 229.64, 325.37,
            332.86, 22.40, 10.50, 9.05, 792.54, 33.04, 2223.85, 142.44, 5.87, 12.57, 4.46, 9.50, 6.06]
}

df = pd.DataFrame(data)

def assign_color(vif):
    if vif < 5:
        return 'green'
    elif 5 <= vif <= 10:
        return 'orange'
    else:
        return 'red'

df['color'] = df['VIF'].apply(assign_color)

fig_vif = px.bar(df, x="feature", y="VIF", color="color",color_discrete_map={"green": "green", "orange": "orange", "red": "red"},title="VIF Histogram con Colorazione per Soglie")
fig_vif.update_layout(xaxis_tickangle=-45, width=1200, height=600)
fig_vif.show()

#### Correlazione Features con Target

In [ ]:
abbreviated_columns = [
    'bnb_close', 'doge_close', 'eth_close', 'sol_close', 'xrp_close',
    'cattle_close', 'corn_close', 'crude_close', 'gold_close', 'silver_close',
    'soybeans_close', 'wheat_close',
    'cac_close', 'dax_close', 'dow_close', 'euro_close', 'ftse_close',
    'ibov_close', 'ipc_close', 'nasdaq_close', 'russell_close', 'sp_close', 'tsx_close', 'vix_close',
    'funding_rate', 'fear_greed', 'g_trend_buy_crypto', 'g_trend_bitcoin'
]

In [ ]:
corr_matrix

,BNB_USDT_1h_close,BTC_USDT_1h_close,DOGE_USDT_1h_close,ETH_USDT_1h_close,SOL_USDT_1h_close,XRP_USDT_1h_close,cattle_Close LE=F,corn_Close ZC=F,crude_Close CL=F,gold_Close GC=F,...,IPC_Close ^MXX,NASDAQ_Close ^IXIC,Russell_Close ^RUT,S&P_Close ^GSPC,S&P_Close ^GSPTSE,VIX_Close ^VIX,funding_rate,fear_gread_index,google_trends_buy_crypto,google_trends_bitcoin
BNB_USDT_1h_close,1.000000,0.929371,0.765242,0.763460,0.922498,0.562355,0.559891,-0.516887,-0.331397,0.860347,...,-0.156359,0.898196,0.819578,0.912731,0.895053,0.180449,0.485636,0.156813,0.541157,0.464900
BTC_USDT_1h_close,0.929371,1.000000,0.879850,0.706120,0.933759,0.758994,0.690843,-0.487790,-0.399215,0.881107,...,-0.242002,0.928724,0.834951,0.932430,0.931713,0.196772,0.259420,0.217048,0.689247,0.609624
DOGE_USDT_1h_close,0.765242,0.879850,1.000000,0.653482,0.813519,0.765566,0.536293,-0.256015,-0.331119,0.662112,...,-0.343219,0.769906,0.758964,0.758445,0.804961,0.044773,0.111167,0.376813,0.850372,0.698477
ETH_USDT_1h_close,0.763460,0.706120,0.653482,1.000000,0.836517,0.233684,0.250993,-0.536515,-0.014220,0.390828,...,0.105564,0.696238,0.701658,0.674940,0.545476,-0.280931,0.596895,0.547000,0.602453,0.561962
SOL_USDT_1h_close,0.922498,0.933759,0.813519,0.836517,1.000000,0.530634,0.532110,-0.634096,-0.283419,0.764615,...,-0.106299,0.920904,0.889701,0.924090,0.865380,0.033267,0.494668,0.342558,0.680590,0.602353
XRP_USDT_1h_close,0.562355,0.758994,0.765566,0.233684,0.530634,1.000000,0.718844,-0.020590,-0.414434,0.703225,...,-0.392550,0.620207,0.498465,0.611258,0.719741,0.284248,-0.359561,-0.017569,0.595008,0.411916
cattle_Close LE=F,0.559891,0.690843,0.536293,0.250993,0.532110,0.718844,1.000000,-0.390450,-0.126619,0.699534,...,-0.416182,0.658479,0.498224,0.658736,0.664818,0.351845,-0.067004,-0.115369,0.375574,0.281847
corn_Close ZC=F,-0.516887,-0.487790,-0.256015,-0.536515,-0.634096,-0.020590,-0.390450,1.000000,-0.014500,-0.439824,...,0.069057,-0.634924,-0.596794,-0.641547,-0.468762,-0.036888,-0.604317,-0.079112,-0.215757,-0.284122
crude_Close CL=F,-0.331397,-0.399215,-0.331119,-0.014220,-0.283419,-0.414434,-0.126619,-0.014500,1.000000,-0.525082,...,0.118280,-0.360996,-0.360910,-0.387596,-0.498357,-0.327821,0.121782,0.052550,-0.253432,-0.238591
gold_Close GC=F,0.860347,0.881107,0.662112,0.390828,0.764615,0.703225,0.699534,-0.439824,-0.525082,1.000000,...,-0.313937,0.852227,0.727572,0.883287,0.934880,0.462288,0.219011,-0.133848,0.411714,0.349805


In [ ]:
# Calcolo della matrice di correlazione
corr_matrix = df_lstm.corr()
target_corr = corr_matrix['BTC_USDT_1h_close'].drop('BTC_USDT_1h_close')
target_corr_sorted = target_corr.reindex(target_corr.abs().sort_values(ascending=False).index)

corr_df = pd.DataFrame({
    'Variabile': target_corr_sorted.index,
    'Correlazione': target_corr_sorted.values,
    'Variabile Abbreviata': abbreviated_columns
})


fig = px.bar(corr_df,x='Correlazione',y='Variabile',
    orientation='h',
    color='Correlazione', color_continuous_scale='RdBu')

fig.update_layout(title={'text': "Correlazione delle feature con BTC_USDT_1h_close",'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'},},
                  yaxis=dict(autorange="reversed"),  height=800, width=1200, )
fig.show()

In [ ]:
fig.write_image('Correlazione_target_features.pdf')

In [ ]:
significant_features = target_corr[abs(target_corr) > 0.1].index.tolist()
print("Variabili significative:", significant_features)

Variabili significative: ['BNB_USDT_1h_close', 'DOGE_USDT_1h_close', 'ETH_USDT_1h_close', 'SOL_USDT_1h_close', 'XRP_USDT_1h_close', 'cattle_Close LE=F', 'corn_Close ZC=F', 'crude_Close CL=F', 'gold_Close GC=F', 'silver_Close SI=F', 'soybeans_Close ZS=F', 'wheat_Close ZW=F', 'CAC_Close ^FCHI', 'DAX_Close ^GDAXI', 'Dow_Close ^DJI', 'EURO_Close ^STOXX50E', 'FTSE_Close ^FTSE', 'IBOVESPA_Close ^BVSP', 'IPC_Close ^MXX', 'NASDAQ_Close ^IXIC', 'Russell_Close ^RUT', 'S&P_Close ^GSPC', 'S&P_Close ^GSPTSE', 'VIX_Close ^VIX', 'funding_rate', 'fear_gread_index', 'google_trends_buy_crypto', 'google_trends_bitcoin']


Il grafico mostra una forte
correlazione positiva di
Bitcoin con molti asset
azionari ed altre
cryptovalute. Tutte le variabili del
dataset sono inizialmente
considerabili come
significative
(|corr_index|>0.1)

#### Correlazione tra le Features

In [ ]:
features = df_lstm.drop(columns=['BTC_USDT_1h_close'])
correlation_matrix = features.corr()
original_columns = list(correlation_matrix.columns)

In [ ]:
fig_cm = ff.create_annotated_heatmap(z=correlation_matrix.values, x=abbreviated_columns, y=abbreviated_columns,
    annotation_text=np.around(correlation_matrix.values, decimals=2),showscale=True, colorscale='RdBu',
    reversescale=True,
    hoverinfo='text',
)

fig_cm.update_layout(
    title={
        'text': "Correlation Matrix",
        'x': 0.5,
        'xanchor': 'center',  # Ancoraggio al centro
        'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'},
    },
    margin=dict(t=200, b=0, l=0, r=0),
    height=800,
    width=1500,
)
fig_cm.show()

#### PCA

Data la multicollinearità presente tra le varie feature (osservata nella
matrice di correlazione tra le variabili), si è proceduto nella
realizzazione di un PCA (Principal Component Analysis) per ridurre il
numero di feature utilizzate nel modello.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

pca = PCA() #PCA senza ridurre il numero di componenti
X_pca = pca.fit(X_scaled)
explained_variance = X_pca.explained_variance_ratio_ #calcolo la varianza spiegata da ciascuna componente

In [ ]:
# Numero di componenti che spiegano la % della varianza
percentuale_var = 0.95
cumulative_variance = X_pca.explained_variance_ratio_.cumsum()
n_components = (cumulative_variance >= percentuale_var).argmax() + 1
print(f"Numero di componenti che spiegano il {percentuale_var} della varianza: {n_components}")

Numero di componenti che spiegano il 0.95 della varianza: 9


In [ ]:
fig_pca = go.Figure()

# Aggiungiamo la traccia per la "Varianza Spiegata" (sinistra)
fig_pca.add_trace(go.Scatter(x=list(range(1, len(explained_variance) + 1)), y=explained_variance,
                             mode='lines+markers', marker=dict(color='#636EFA'), name='Varianza Spiegata',
                             hovertemplate='%{x}<br>Varianza Spiegata: %{y:.4f}<extra></extra>'))

# Aggiungiamo la traccia per la "Varianza Cumulativa" (destra)
fig_pca.add_trace(go.Scatter(x=list(range(1, len(explained_variance) + 1)), y=explained_variance.cumsum(),
                             mode='lines+markers', marker=dict(color='#EF553B'), name='Varianza Cumulativa',
                             hovertemplate='%{x}<br>Varianza Cumulativa: %{y:.4f}<extra></extra>',
                             yaxis="y2"))

# Linea orizzontale a 95% sulla y2
y_value_percent = percentuale_var * explained_variance.sum()  # 95% del totale della varianza cumulativa
fig_pca.add_trace(go.Scatter(x=[1, len(explained_variance)], y=[y_value_percent, y_value_percent],
                             mode='lines', line=dict(color='gray', dash='dash', width=1), name=f'Linea {round(y_value_percent,4)*100}% Varianza Cumulativa'))

#punto di intersezione tra la curva di varianza cumulativa e la linea orizzontale
cumulative_variance = explained_variance.cumsum()
x_intersection = np.argmax(cumulative_variance >= y_value_percent) + 1  # Aggiungiamo 1 perché gli indici partono da 0
y_intersection = cumulative_variance[x_intersection - 1]  # Otteniamo il valore della varianza cumulativa al punto di intersezione

#Linea verticale perpendicolare (passante per il punto di intersezione)
fig_pca.add_trace(go.Scatter(x=[x_intersection, x_intersection], y=[0, y_intersection],
                             mode='lines', line=dict(color='gray', dash='dash', width=1), name='Linea Perpendicolare'))

fig_pca.update_layout(
    title={'text': "Analisi PCA - Varianza per Componente", 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    showlegend=True, width=1200, height=600,
    yaxis=dict(title="Varianza Spiegata", titlefont=dict(color="#636EFA")),
    yaxis2=dict(title="Varianza Cumulativa", titlefont=dict(color="#EF553B"), overlaying="y", side="right"),
    xaxis=dict(title="Numero di Componente",dtick=1, tickvals=list(range(1, len(explained_variance) + 1)) ),
    legend=dict(x=1.05, xanchor='left', yanchor='middle')
)

fig_pca.show()

In [ ]:
components = X_pca.components_[:n_components] # Prendi i carichi delle prime 'n_components' componenti
var_names = features.columns
pesi_var_pca = pd.DataFrame(components, columns=var_names) #DataFrame con i carichi delle variabili
pesi_var_pca.round(4)
print("Carichi delle variabili per le componenti selezionate:")
pesi_var_pca

Carichi delle variabili per le componenti selezionate:


,BNB_USDT_1h_close,DOGE_USDT_1h_close,ETH_USDT_1h_close,SOL_USDT_1h_close,XRP_USDT_1h_close,cattle_Close LE=F,corn_Close ZC=F,crude_Close CL=F,gold_Close GC=F,silver_Close SI=F,...,IPC_Close ^MXX,NASDAQ_Close ^IXIC,Russell_Close ^RUT,S&P_Close ^GSPC,S&P_Close ^GSPTSE,VIX_Close ^VIX,funding_rate,fear_gread_index,google_trends_buy_crypto,google_trends_bitcoin
0,0.237663,0.203841,0.173823,0.238003,0.169447,0.172561,-0.151493,-0.103857,0.229767,0.220932,...,-0.055042,0.244923,0.227906,0.248130,0.243999,0.045978,0.093927,0.031692,0.158470,0.138746
1,-0.043299,0.003720,-0.335659,-0.139772,0.265840,0.213879,0.175967,-0.196265,0.182772,0.111569,...,-0.331702,-0.019222,-0.106175,-0.010423,0.082352,0.383422,-0.330271,-0.347063,-0.090454,-0.131808
2,-0.031351,0.338765,0.104459,0.055454,0.238681,0.020258,0.203012,-0.033226,-0.096200,-0.131195,...,-0.214213,-0.012838,0.027065,-0.038802,0.010703,-0.094695,-0.234633,0.367209,0.432389,0.367927
3,0.045877,0.052779,-0.007108,-0.036454,0.203576,-0.046337,0.425489,-0.164023,0.038993,0.070974,...,0.365973,-0.043011,-0.029291,-0.037287,0.052459,-0.154847,-0.241084,0.022390,0.000822,-0.055214
4,0.029567,0.018970,0.094659,-0.000991,0.162737,0.418550,-0.048887,0.714582,-0.063001,-0.052349,...,-0.072935,0.003699,-0.130255,-0.020697,-0.071835,-0.101600,-0.100545,-0.081665,-0.030948,-0.091431
5,-0.189750,-0.110363,-0.147191,-0.034449,0.125210,0.166159,-0.194909,-0.138100,-0.025508,-0.270142,...,0.379989,-0.073746,-0.097383,-0.061281,-0.065088,0.155047,-0.331423,-0.013976,0.102064,0.299475
6,0.209278,-0.001671,0.122484,0.052231,-0.136281,-0.129622,0.124524,0.016899,0.103037,0.050785,...,0.116664,-0.185376,-0.246101,-0.134316,-0.058423,0.624454,0.320210,0.040729,0.062240,0.302987
7,0.060204,-0.057610,0.071392,0.077682,0.009828,0.303435,-0.092824,-0.074572,0.092030,0.080258,...,-0.046592,0.003353,-0.113118,-0.009969,-0.035674,0.262722,-0.120265,0.728151,-0.275917,-0.321724
8,-0.146754,-0.007989,-0.172663,-0.071191,-0.086462,0.386245,-0.042912,0.218178,0.034271,0.056792,...,-0.078429,-0.001038,0.130737,0.026727,0.006898,0.189272,0.220658,-0.106629,0.186489,0.310452


In [ ]:
#Le variabili più significative per ogni componente
for i in range(n_components):
    top_features = pesi_var_pca.iloc[i].abs().sort_values(ascending=False).head(5)
    print(f"\nLe variabili più significative per la componente {i+1}:")
    print(top_features)


Le variabili più significative per la componente 1:
S&P_Close ^GSPC       0.248130
Dow_Close ^DJI        0.247191
NASDAQ_Close ^IXIC    0.244923
S&P_Close ^GSPTSE     0.243999
SOL_USDT_1h_close     0.238003
Name: 0, dtype: float64

Le variabili più significative per la componente 2:
VIX_Close ^VIX       0.383422
fear_gread_index     0.347063
ETH_USDT_1h_close    0.335659
IPC_Close ^MXX       0.331702
funding_rate         0.330271
Name: 1, dtype: float64

Le variabili più significative per la componente 3:
google_trends_buy_crypto    0.432389
google_trends_bitcoin       0.367927
fear_gread_index            0.367209
DOGE_USDT_1h_close          0.338765
IBOVESPA_Close ^BVSP        0.255465
Name: 2, dtype: float64

Le variabili più significative per la componente 4:
corn_Close ZC=F     0.425489
CAC_Close ^FCHI     0.423729
wheat_Close ZW=F    0.400204
IPC_Close ^MXX      0.365973
funding_rate        0.241084
Name: 3, dtype: float64

Le variabili più significative per la componente 5:
crud

In [ ]:
X_pca = pca.transform(X_scaled)  # X_scaled è il tuo dataset normalizzato

# Crea un DataFrame con le componenti principali
pca_df = pd.DataFrame(X_pca[:, :n_components], columns=[f'PC{i+1}' for i in range(n_components)])
pca_df['BTC_USDT_1h_close'] = df_lstm['BTC_USDT_1h_close'].values
pca_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,BTC_USDT_1h_close
0,-5.402583,0.246620,1.256260,2.724990,0.156880,-1.748157,1.610140,-0.081639,0.123379,30277.44
1,-5.404257,0.245995,1.254093,2.724101,0.156254,-1.747692,1.610205,-0.081480,0.123903,30240.00
2,-5.402308,0.245686,1.255573,2.724239,0.156316,-1.748281,1.610484,-0.081352,0.123539,30267.06
3,-5.403814,0.245968,1.254817,2.723514,0.155280,-1.748418,1.610453,-0.081967,0.124735,30248.96
4,-5.396439,0.241522,1.261578,2.725036,0.157751,-1.752490,1.612810,-0.081161,0.120859,30302.08


In [ ]:
pca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17515 entries, 0 to 17514
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PC1                17515 non-null  float64
 1   PC2                17515 non-null  float64
 2   PC3                17515 non-null  float64
 3   PC4                17515 non-null  float64
 4   PC5                17515 non-null  float64
 5   PC6                17515 non-null  float64
 6   PC7                17515 non-null  float64
 7   PC8                17515 non-null  float64
 8   PC9                17515 non-null  float64
 9   BTC_USDT_1h_close  17515 non-null  float64
dtypes: float64(10)
memory usage: 1.3 MB


In [ ]:
X_pca = pca_df.drop(columns=['BTC_USDT_1h_close'])
#ANALISI VIF
X_pca_const = sm.add_constant(X_pca)

# Calcolo VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_pca.columns
vif_data["VIF"] = [variance_inflation_factor(X_pca_const.values, i+1) for i in range(len(X_pca.columns))]  # i+1 per saltare la costante

print(vif_data)

  feature  VIF
0     PC1  1.0
1     PC2  1.0
2     PC3  1.0
3     PC4  1.0
4     PC5  1.0
5     PC6  1.0
6     PC7  1.0
7     PC8  1.0
8     PC9  1.0


In [ ]:
#Verifica se ha mantenuto l'ordine delle righe
print(df_lstm['BTC_USDT_1h_close'].head(-5))
print(pca_df['BTC_USDT_1h_close'].head(-5))

0        30277.44
1        30240.00
2        30267.06
3        30248.96
4        30302.08
           ...   
17505    84454.10
17506    84658.00
17507    84929.80
17508    85136.30
17509    84382.30
Name: BTC_USDT_1h_close, Length: 17510, dtype: float64
0        30277.44
1        30240.00
2        30267.06
3        30248.96
4        30302.08
           ...   
17505    84454.10
17506    84658.00
17507    84929.80
17508    85136.30
17509    84382.30
Name: BTC_USDT_1h_close, Length: 17510, dtype: float64


### 2.2. Train e Test

In [ ]:
df_ridotto = pca_df

In [ ]:
giorno = 24
settimana = 24 * 7
mese = 24 * 30
sei_mesi = 24 * 30 * 6
anno = 24 * 30 *12
due_anni =24 * 30 *12 *2

In [ ]:
#Definizione del Train e Test Set
train_size = int(len(df_ridotto) * 0.8) #dimensione del train. Prendiamo l'80% del dataframe
train, test = df_ridotto.iloc[:train_size], df_ridotto.iloc[train_size:] #per il train prendiamo dal df, dal primo valore fino all'ultimo valore di train_size.
#                                                                         Mentre per il test, prendiamo dal valore di tran_size fino alla fine (20%)

#per avere come indice la data, così da metterla nell'asse X del plot
#ricava dal dataset il datatime, usando le posizioni per creare il test e train dates
train_dates = df_lstm_with_dt['Datetime'].iloc[:train_size]
test_dates = df_lstm_with_dt['Datetime'].iloc[train_size:]

#Plot del train e test set
def plot_time_series_plotly(train, test, train_dates, test_dates, title="Time Series: divisione del dataframe in train e test"):
    fig = go.Figure()
    spessore = 2
    fig.add_trace(go.Scatter(x=train_dates, y=train['BTC_USDT_1h_close'],mode='lines',name='Train', line=dict(width=spessore),
                             hovertemplate='%{x|%Y-%m-%d}<br>Prezzo: %{y:.2f}<extra></extra>'))
    fig.add_trace(go.Scatter(x=test_dates, y=test['BTC_USDT_1h_close'],mode='lines',name='Test', line=dict(width=spessore),
                             hovertemplate='%{x|%Y-%m-%d}<br>Prezzo: %{y:.2f}<extra></extra>'))
    fig.update_layout(title={'text': title,'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'},},
                      xaxis_title='Date',yaxis_title='Price',
                      width=1200,height=700)
    fig.show()

plot_time_series_plotly(train[['BTC_USDT_1h_close']],test[['BTC_USDT_1h_close']],train_dates,test_dates)

Abbiamo preparato il test e train set per un'analisi predittiva (timeseries) con una previsione di una settimana

In [ ]:
target_column = 'BTC_USDT_1h_close'
sequence_length = settimana  # una settimana se dati orari
features = [col for col in df_ridotto.columns if col != target_column]

train_size = int(len(df_ridotto) * 0.8)
train = df_ridotto.iloc[:train_size]
test = df_ridotto.iloc[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train[features + [target_column]])
test_scaled = scaler.transform(test[features + [target_column]])

def create_sequences_multivariate(data, sequence_length, target_column_index):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :-1])  # tutte le feature tranne la target
        y.append(data[i + sequence_length, target_column_index])  # target alla fine
    return np.array(X), np.array(y)

target_index = len(features)

X_train, y_train = create_sequences_multivariate(train_scaled, sequence_length, target_index)
X_test, y_test = create_sequences_multivariate(test_scaled, sequence_length, target_index)

print("--- Controllo finale ---")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

--- Controllo finale ---
X_train shape: (13844, 168, 9)
y_train shape: (13844,)
X_test shape: (3335, 168, 9)
y_test shape: (3335,)


Non da usare

```
# Funzione per creare una sequenza per LSTM - Analisi Monovariata
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y) #trasformiamo in np array

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train[['BTC_USDT_1h_close']])
test_scaled = scaler.transform(test[['BTC_USDT_1h_close']])

#Lunghezza della predizione
sequence_length = settimana

# Creazione delle sequenza con la funzione
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_test, y_test = create_sequences(test_scaled, sequence_length)

# Reshape input to be [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
```



In [ ]:
X_train.shape

(13844, 168, 9)

In [ ]:
X_train.shape
"""
X_train.shape = (13844, 168, 28) significa che ora stai correttamente usando:
	•	13844 campioni (finestre temporali)
	•	168 timestep (quindi 1 settimana di dati orari)
	•	28 feature per ogni timestep (quindi 28 colonne usate come input)
    """

'\nX_train.shape = (13844, 168, 28) significa che ora stai correttamente usando:\n\t•\t13844 campioni (finestre temporali)\n\t•\t168 timestep (quindi 1 settimana di dati orari)\n\t•\t28 feature per ogni timestep (quindi 28 colonne usate come input)\n    '

In [ ]:
lunghezza_sequenz = X_train.shape[1]
n_features = X_train.shape[2]
print(f"Lunghezza della sequenza: {lunghezza_sequenz}")
print(f"Numero di feature: {n_features}")

Lunghezza della sequenza: 168
Numero di feature: 9


### 2.3. Modello LSTM - Autoregressivo

Tra le varie opzioni di modelli di machine learning o reti neurali, la
scelta è ricaduta su Long Short-Term Memory layer model (LSTM). Tale
decisione è stata effettuata in quanto i dati a disposizione sono di tipo
temporale e non sarebbe stato accurato utilizzare altri modelli di
Machine Learning.
I dati utilizzati saranno esclusivamente i dati di mercato «close» per
evitare inutili ridondanze.


Questo modello  fa previsioni su base oraria, e per fare ogni previsione ha bisogno di una “finestra” di sequence_length = 168 ore precedenti (1 settimana). Quindi  il modello  non produce previsioni per i primi sequence_length, perché ha bisogno di quella finestra come input per la prima previsione.

Ad esempio:
- Se sequence_length = 168, la prima previsione sarà in corrispondenza del dato 168.
- Quindi i primi 168 valori reali non vanno confrontati con nulla, perché non c’è ancora una previsione.



In [ ]:
"""
# Define custom metrics
def R2Score(y_true, y_pred):
    SS_res =  tf.reduce_sum(tf.square( y_true-y_pred ))
    SS_tot = tf.reduce_sum(tf.square( y_true - tf.reduce_mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + tf.keras.backend.epsilon()) )

def RootMeanSquaredError(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


# Load the model with custom objects
model = load_model("/content/modello_lstm_completo.keras",
                   custom_objects={'R2Score': R2Score,
                                   'RootMeanSquaredError': RootMeanSquaredError})
"""
model = load_model("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/modello_completo.keras")
with open("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/history.pkl", "rb") as file_pi:
    history = pickle.load(file_pi)

y_pred_scaled = model.predict(X_test)
y_test_reshaped = np.zeros((y_test.shape[0], train_scaled.shape[1]))
y_test_reshaped[:, target_index] = y_test

y_test_actual = scaler.inverse_transform(y_test_reshaped)[:, target_index]

y_pred_reshaped = np.zeros((y_pred_scaled.shape[0], train_scaled.shape[1]))
y_pred_reshaped[:, target_index] = y_pred_scaled.flatten()
y_pred = scaler.inverse_transform(y_pred_reshaped)[:, target_index]

lstm_mse = mean_squared_error(y_test_actual, y_pred)
lstm_mae = mean_absolute_error(y_test_actual, y_pred)
lstm_r2 = r2_score(y_test_actual, y_pred)
lstm_loss = model.evaluate(X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 



105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0508 - r2_score: -13.3663 - root_mean_squared_error: 0.2181


#### Modello

In [ ]:
tensorflow.keras.backend.clear_session()

In [ ]:
# Definizione del modello
tensorflow.random.set_seed(42)

model = Sequential()

Reg_l1 = 0.00
Reg_l2 = 0.00
DP_out = 0.5

#Input Layer
model.add(Input(shape=(sequence_length, n_features)))  # Input esplicito

#Hidden Layers
#model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
#model.add(Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=Reg_l1, l2=Reg_l2)))
#model.add(Dropout(DP_out))
model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=Reg_l1, l2=Reg_l2)))
#model.add(Dropout(0.3))
model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=Reg_l1, l2=Reg_l2)))

#Output Layers
model.add(Dense(1, kernel_regularizer=l1_l2(l1=Reg_l1, l2=Reg_l2)))

# Compilazione
model.compile(optimizer='adam',
              loss='mse',
              metrics=[tensorflow.keras.metrics.R2Score(), tensorflow.keras.metrics.RootMeanSquaredError()])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,569 (84.25 KB)

 Trainable params: 21,569 (84.25 KB)

 Non-trainable params: 0 (0.00 B)

Per l’addestramento del
modello è stato inserito un
EarlyStopg in caso la
variazione marginale della
value loss sia inferiore a
0,00001, fino ad un massimo di
ducecento epoche e una
batch_size di trenta.

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=1e-5,patience=5,verbose=1,mode='auto',restore_best_weights=True)
epoche = 50
n_batch_size = 30

In [ ]:
print(f"I casi presenti nel dataset:{len(df_lstm-1)}, \n essendo che la batch size è di {n_batch_size}, serviranno {len(df_lstm-1)/n_batch_size} passaggi per completare un'epoca ")

I casi presenti nel dataset:17515, 
 essendo che la batch size è di 30, serviranno 583.8333333333334 passaggi per completare un'epoca 


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=epoche, batch_size=n_batch_size,
                    verbose=1, validation_split=0.1,
                    callbacks=[early_stop
                               ])

Epoch 1/50
416/416 ━━━━━━━━━━━━━━━━━━━━ 33s 71ms/step - loss: 0.0162 - r2_score: 0.7075 - root_mean_squared_error: 0.1138 - val_loss: 0.0021 - val_r2_score: 0.8808 - val_root_mean_squared_error: 0.0456
Epoch 2/50
416/416 ━━━━━━━━━━━━━━━━━━━━ 30s 71ms/step - loss: 3.0760e-04 - r2_score: 0.9945 - root_mean_squared_error: 0.0175 - val_loss: 0.0012 - val_r2_score: 0.9285 - val_root_mean_squared_error: 0.0353
Epoch 3/50
416/416 ━━━━━━━━━━━━━━━━━━━━ 43s 76ms/step - loss: 2.5468e-04 - r2_score: 0.9954 - root_mean_squared_error: 0.0160 - val_loss: 0.0011 - val_r2_score: 0.9361 - val_root_mean_squared_error: 0.0334
Epoch 4/50
416/416 ━━━━━━━━━━━━━━━━━━━━ 39s 70ms/step - loss: 2.2062e-04 - r2_score: 0.9960 - root_mean_squared_error: 0.0148 - val_loss: 9.3901e-04 - val_r2_score: 0.9462 - val_root_mean_squared_error: 0.0306
Epoch 5/50
416/416 ━━━━━━━━━━━━━━━━━━━━ 57s 109ms/step - loss: 1.9487e-04 - r2_score: 0.9965 - root_mean_squared_error: 0.0140 - val_loss: 7.5095e-04 - val_r2_score: 0.9570 - v

In [ ]:
# Salva modello e history
model.save("modello_completo.keras")
with open("history.pkl", "wb") as file_pi:
    pickle.dump(history.history, file_pi)

# === In futuro ===
"""
# Carica modello e history
from tensorflow.keras.models import load_model
import pickle

model = load_model("modello_completo.h5")
with open("history.pkl", "rb") as file_pi:
    loaded_history = pickle.load(file_pi)
"""

'\n# Carica modello e history\nfrom tensorflow.keras.models import load_model\nimport pickle\n\nmodel = load_model("modello_completo.h5")\nwith open("history.pkl", "rb") as file_pi:\n    loaded_history = pickle.load(file_pi)\n'

Spiegazione codice:

```
y_pred_scaled = model.predict(X_test)
```
- model.predict(X_test) esegue la predizione sul set di test (X_test) usando il modello LSTM già addestrato.
- Il risultato (y_pred_scaled) è scalato, cioè normalizzato (per esempio tra 0 e 1).



```
y_test_reshaped = np.zeros((y_test.shape[0], train_scaled.shape[1]))  
y_test_reshaped[:, target_index] = y_test  
```
- y_test contiene i valori reali da confrontare con le predizioni.
- Viene creato un array pieno di zeri della stessa lunghezza dei dati di test, ma con tutte le colonne presenti nel set scalato originale (train_scaled.shape[1]).
- Si riempie solo la colonna dell’indice target (target_index) con i valori reali (y_test), lasciando le altre colonne a 0. Questo serve per poter usare lo scaler inverso, che si aspetta un array con tutte le feature originali.



```
y_test_actual = scaler.inverse_transform(y_test_reshaped)[:, target_index]
```
- Si esegue la trasformazione inversa dello scaler per riportare i dati nella scala originale.
- Poi si estrae solo la colonna (target_index), che è il vero valore reale nella scala originale.




```
y_pred_reshaped = np.zeros((y_pred_scaled.shape[0], train_scaled.shape[1]))
y_pred_reshaped[:, target_index] = y_pred_scaled.flatten()  
y_pred = scaler.inverse_transform(y_pred_reshaped)[:, target_index]
```
Anche le predizioni vengono invertite nella scala originale, e si estrae solo la colonna di interesse.






In [ ]:
y_pred_scaled = model.predict(X_test) #y scalato da prima, con tecnica MinMax

# Ricostruzione del formato per l’inverso dello scaling
y_test_reshaped = np.zeros((y_test.shape[0], train_scaled.shape[1]))
y_test_reshaped[:, target_index] = y_test

y_test_actual = scaler.inverse_transform(y_test_reshaped)[:, target_index]

y_pred_reshaped = np.zeros((y_pred_scaled.shape[0], train_scaled.shape[1]))
y_pred_reshaped[:, target_index] = y_pred_scaled.flatten()
y_pred = scaler.inverse_transform(y_pred_reshaped)[:, target_index]


lstm_mse = mean_squared_error(y_test_actual, y_pred)
lstm_mae = mean_absolute_error(y_test_actual, y_pred)
lstm_r2 = r2_score(y_test_actual, y_pred)
lstm_loss = model.evaluate(X_test, y_test)

print(f"LSTM Forecast - MSE: {lstm_mse}, MAE: {lstm_mae}, R2: {lstm_r2}, Loss: {lstm_loss}")

105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0508 - r2_score: -13.3663 - root_mean_squared_error: 0.2181
LSTM Forecast - MSE: 355522534.22333676, MAE: 18056.817249111427, R2: -5.2044260493606815, Loss: [0.07530300319194794, -5.204392433166504, 0.27441391348838806]


#### Grafici output

In [ ]:
#@title Loss, MSE, e R² - Test vs Validation
#Per modelli caricati usare i primi (history), per modelli addestrati sul momemnto (history.history)

loss = history['loss']
val_loss = history['val_loss']
r2 = history['r2_score']
val_r2_score = history['val_r2_score']
root_mean_squared_error = history['root_mean_squared_error']
val_root_mean_squared_error = history['val_root_mean_squared_error']
"""

loss = history.history['loss']
val_loss = history.history['val_loss']
r2 = history.history['r2_score']
val_r2_score = history.history['val_r2_score']
root_mean_squared_error = history.history['root_mean_squared_error']
val_root_mean_squared_error = history.history['val_root_mean_squared_error']
"""

fig_lstm = make_subplots(rows=2, cols=2,vertical_spacing=0.15,subplot_titles=("Loss and Validation Loss", "MSE", "R2"), row_heights=[0.45, 0.45])
fig_lstm.add_trace(go.Scatter(x=list(range(len(loss))), y=loss, mode='lines', name='Loss'),row=1, col=1)
fig_lstm.add_trace(go.Scatter(x=list(range(len(val_loss))), y=val_loss, mode='lines', name='Validation Loss'),row=1, col=1)
fig_lstm.add_trace(go.Scatter(x=list(range(len(r2))), y=r2, mode='lines', name='R2'),row=2, col=1)
fig_lstm.add_trace(go.Scatter(x=list(range(len(val_r2_score))), y=val_r2_score, mode='lines', name='Validation R2'),row=2, col=1)
fig_lstm.add_trace(go.Scatter(x=list(range(len(root_mean_squared_error))), y=root_mean_squared_error, mode='lines', name='Root Mean Squared Error'),row=1, col=2)
fig_lstm.add_trace(go.Scatter(x=list(range(len(val_root_mean_squared_error))), y=val_root_mean_squared_error, mode='lines', name='Val Root Mean Squared Error'),row=1, col=2)

fig_lstm.update_layout(
    title= {'text': "Confronto Metriche di Performance: Loss, MSE, e R² - Test vs Validation",'x': 0.5, 'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Epochs",showlegend=True,width=1500,height=800,template='plotly_dark' )
fig_lstm.show()

Nonostante i valori eccellenti sul test set, sul validation set i valori di Loss, MSE e R2 prensenta risultati peggiori, il che suggerisce la presenza di overfitting.
Inoltre,tra la 15esima e 20esima epoca il modello tende a peggiorare.

In [ ]:
def plot_forecasts_plotly(test, actual_col, pred, test_dates, titolo):
    index = test_dates[sequence_length:] #Estrae le date corrispondenti alle previsioni. Salta le prime sequence_length date, perché non hai previsioni per quei giorni.
    actual = test[actual_col].values[sequence_length:] #Estrae i valori reali dal test, nella colonna 'BTC_USDT_1h_close'. Anche qui, salta i primi sequence_length dati, per restare allineato con le previsioni.

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=index, y=actual, mode='lines', name='Actual', line=dict(color='white')))
    fig.add_trace(go.Scatter(x=index, y=pred, mode='lines', name='Forecast', line=dict(color='lightsalmon')))

    fig.update_layout(
        title= {'text': titolo, 'x': 0.5, 'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
        xaxis_title="Date",
        yaxis_title="Price",
        legend=dict(x=0, y=1),
        template='plotly_dark',
        width=1200,
        height=700
    )
    fig.show()

In [ ]:
plot_forecasts_plotly(test, 'BTC_USDT_1h_close', y_pred, test_dates, 'Forecast Comparison')

In [ ]:
scarto = np.abs(y_test_actual - y_pred)
media_scarto = np.mean(scarto)
print("Lo scarto medio tra la Y reale e la predetta è di:",round(media_scarto, 2), "dollari")

y_corretta = y_pred + media_scarto
print("\nLa nuova y_corretta:", y_corretta, "\n\nLa y_pred:", y_pred)

Lo scarto medio tra la Y reale e la predetta è di: 18056.82 dollari

La nuova y_corretta: [102938.61974921 103042.92817818 103072.6369749  ...  75959.82759179
  75866.74494333  75843.95550311] 

La y_pred: [84881.8025001  84986.11092906 85015.81972579 ... 57903.01034268
 57809.92769422 57787.138254  ]


In [ ]:
def plot_forecasts_plotly(test, actual_col, pred, test_dates, titolo, sequence_length):
    index = test_dates[sequence_length:]
    actual = test[actual_col].values[sequence_length:]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=index, y=actual, mode='lines', name='Actual', line=dict(color='white')))
    fig.add_trace(go.Scatter(x=index, y=pred, mode='lines', name='Forecast_corretta', line=dict(color='deepskyblue')))

    fig.update_layout(
        title={'text': titolo, 'x': 0.5, 'xanchor': 'center','font': {'size': 20, 'family': 'Arial'}},
        xaxis_title="Date",
        yaxis_title="Price",
        legend=dict(x=0, y=1),
        template='plotly_dark',
        width=1200,
        height=700
    )

    return fig

In [ ]:
fig = plot_forecasts_plotly(test, 'BTC_USDT_1h_close', y_corretta, test_dates, 'Forecast Comparison, con y_corretta',sequence_length)
fig.show()

In [ ]:
#@title Previsioni nell'ultima settimana
#Le previsioni (y_corretta) iniziano da t = 168 in poi, in quanto le prime 168 ore non hanno previsioni (servono solo come input)
#quindi bisogna SFASARLI, se no i dati reali e predetti non sarebbero allineati
start_idx_test = -(2 * sequence_length) # = -336, prendi le ultime 336 ore (2 settimane) dai dati reali.
start_idx_y = -(1 * sequence_length) # = -168, prendi solo l’ultima settimana di previsioni.

#Ultime 336 ore: [---INPUT---][-----------PREVISIONI----------]
#                (168 ore)    (168 previsioni e dati reali)

test_week = test.iloc[start_idx_test:]
test_dates_week = test_dates[start_idx_test:]
y_corretta_week = y_corretta[start_idx_y:]


plot_forecasts_plotly(test_week, 'BTC_USDT_1h_close', y_corretta_week, test_dates_week, "Forecast Focus: nell'ultima settimana con y_corretta")

In [ ]:
fig_learning_curve.write_image("Learning_Curve_GB_RedM.pdf")

Con la correzzione del target (y_corretta), il modello predice abbastanza bene l'andamento del prezzo di chiusura del Bitcoin. In dettaglio da quest'ultimo grafico, che fa un focus sulle prime 100 ore, il modello predice bene la tendenza del prezzo.

In [ ]:
def plot_r2_scatter(test, actual_col, pred, test_dates):
    index = test_dates[sequence_length:]
    actual = test[actual_col].values[sequence_length:]

    min_val = min(actual.min(), pred.min())
    max_val = max(actual.max(), pred.max())

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=actual,y=pred, mode='markers', name='Predizioni',marker=dict(size=6, opacity=0.6, color='lightsalmon'))) #punti
    fig.add_trace(go.Scatter( x=[min_val, max_val], y=[min_val, max_val], mode='lines',name='y = x',line=dict(color='white', dash='dash'))) # Linea ideale: predicted = actual
    fig.update_layout(title={'text': f'R2 Scatter Plot: {round(r2_score(y_test_actual, pred),2)}', 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},xaxis_title='Actual',yaxis_title='Predicted',
        width=600,height=600,legend=dict(x=0, y=1),template='plotly_dark')
    fig.show()

#plot_r2_scatter(test, 'BTC_USDT_1h_close', y_pred, test_dates)
#plot_r2_scatter(test, 'BTC_USDT_1h_close', y_corretta, test_dates)

In [ ]:
def plot_r2_subplots(test, actual_col, preds, labels, test_dates):
    index = test_dates[sequence_length:]
    actual = test[actual_col].values[sequence_length:]
    colors = ['lightsalmon', 'deepskyblue', 'lightgreen', 'gold', 'orchid', 'lightcoral']

    fig = make_subplots(rows=1, cols=len(preds),subplot_titles=[f"{label} (R²: {round(r2_score(actual, pred), 2)})" for label, pred in zip(labels, preds)])

    for i, (pred, label) in enumerate(zip(preds, labels)):
        min_val = min(actual.min(), pred.min())
        max_val = max(actual.max(), pred.max())
        show_legend = (i == 0)  # per non avere una doppia leggenda per la linea

        #punti
        fig.add_trace(go.Scatter(x=actual,y=pred,mode='markers', name=f'{label} - Predizioni', marker=dict(size=6, opacity=0.6, color=colors[i % len(colors)]),
            showlegend=True), row=1, col=i+1)

        # Linea y = x
        fig.add_trace(go.Scatter(x=[min_val, max_val],y=[min_val, max_val],mode='lines',name='y = x',line=dict(color='white', dash='dash'),showlegend=show_legend), row=1, col=i+1)

        fig.update_xaxes(title_text="Actual", row=1, col=i+1)
        fig.update_yaxes(title_text="Predicted", row=1, col=i+1)

    fig.update_layout(title={'text': 'Confronto Scatter Plot R² tra modelli','x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
        width=1200,height=600,legend=dict(x=0, y=1),template='plotly_dark')

    fig.show()

In [ ]:
plot_r2_subplots(
    test=test,
    actual_col='BTC_USDT_1h_close',
    preds=[y_pred, y_corretta],
    labels=["Modello 1", "Modello 2 corretto"],
    test_dates=test_dates
)

In [ ]:
def plot_pct_change(data, col, dates):
    index = test_dates[sequence_length:]
    pct = data.pct_change()[col].values
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=index, y=pct, mode='lines', name='Variazione %', line=dict(), hovertemplate='%{x}<br>Variazione: %{y:.2%}<extra></extra>'))
    fig.add_trace(go.Scatter(x=index, y=[0]*len(index), mode='lines', name='Linea a y=0', line=dict(color='white', width=1), showlegend=True))
    fig.update_layout(
        title={'text': 'BTC/USDT - Variazione % del prezzo di chiusura in base al valore precedente', 'x': 0.5, 'xanchor': 'center',
               'font': {'size': 15, 'family': 'Arial', 'weight': 'bold'}},
        xaxis_title="Data/Ora",yaxis_title="Variazione %", yaxis=dict(tickformat=".0%"),
        legend=dict(x=0, y=1),width=600,height=600,template='plotly_dark'
    )
    fig.show()
    return fig

fig = plot_pct_change(test, 'BTC_USDT_1h_close', test.index)

#### Predizione per le prossime due settimane

In [ ]:
n_hours_to_predict = 2 * 24 * 7  # Numero di ore da predire (1 settimana)

# Ultima sequenza completa (include target)
last_sequence = train_scaled[-sequence_length:]  # shape: (sequence_length, n_features + 1)

predictions = []

for _ in range(n_hours_to_predict):
    # Prepara l'input per il modello (rimuovi la colonna target)
    last_sequence_reduced = last_sequence[:, :-1]  # shape: (sequence_length, n_features)

    # Predizione
    next_pred = model.predict(last_sequence_reduced[np.newaxis, :, :], verbose=0)
    predictions.append(next_pred[0, 0])

    # Costruisci il nuovo timestep
    new_timestep = last_sequence[-1].copy()  # Prendi l'ultimo timestep intero
    new_timestep[target_index] = next_pred[0, 0]  # Inserisci la nuova previsione nella colonna target

    # Aggiorna la sequenza (sliding window)
    last_sequence = np.concatenate([last_sequence[1:], new_timestep[np.newaxis, :]], axis=0)

In [ ]:
# Ricostruzione per inverse_transform
predictions_reshaped = np.zeros((len(predictions), train_scaled.shape[1]))
predictions_reshaped[:, target_index] = predictions  # Solo la colonna della target

# Inverso dello scaling
predictions = scaler.inverse_transform(predictions_reshaped)[:, target_index]

In [ ]:
n_predictions = len(predictions)
start_date = datetime(2025, 4, 15)
date_range = [start_date + timedelta(hours=i) for i in range(n_predictions)] # Lista di timestamp per ogni ora

df_pred = pd.DataFrame({'datetime': date_range,'prediction': predictions})
df_pred.head()

,datetime,prediction
0,2025-04-15 00:00:00,88930.307955
1,2025-04-15 01:00:00,89111.493118
2,2025-04-15 02:00:00,89233.186967
3,2025-04-15 03:00:00,89311.497127
4,2025-04-15 04:00:00,89358.517626


In [ ]:
fig_prev = go.Figure()
fig_prev.add_trace(go.Scatter(x=date_range,y=predictions,mode='lines',name='Forecast',line=dict(color='lightsalmon')))
fig_prev.update_layout(title={'text': "Forecast - Prossime 2 Settimane",'x': 0.5,'xanchor': 'center',
                              'font': {'size': 20,'family': 'Arial','weight': 'bold'}},
    xaxis_title="Date",yaxis_title="Price",legend=dict(x=0, y=1),template='plotly_dark',width=1200,height=700)
fig_prev.show()

### 2.4. Modello LSTM multi-step

Il modello autoregressivo attualmente utilizzato (par. 2.3) effettua previsioni passo dopo passo, utilizzando ogni output come input per la previsione successiva.
Questo approccio, tuttavia, comporta un effetto cumulativo degli errori: con il passare del tempo, il modello tende a “perdere il contatto” con la realtà, come si osserva chiaramente nel grafico relativo alle previsioni su due settimane.

In conclusione, il modello evidenzia una leggera tendenza al rialzo per il prezzo di chiusura del BTC nelle prossime due settimane.

#### Preparazione (train e test)

In [ ]:
def create_sequences_multi_output(data, sequence_length, n_future, target_index):
    X, y = [], []
    for i in range(len(data) - sequence_length - n_future):
        X.append(data[i:i + sequence_length, :-1])  # features
        y.append(data[i + sequence_length:i + sequence_length + n_future, target_index])  # multi-target
    return np.array(X), np.array(y)

In [ ]:
# Parametri
sequence_length_multi = 24 * 7  # una settimana (input)
n_future = 24 * 14        # due settimane (output)
target_column = 'BTC_USDT_1h_close'
features_multi = [col for col in df_ridotto.columns if col != target_column]
target_multi_index = len(features_multi)  # target è l'ultima colonna

# Normalizzazione
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train[features_multi + [target_column]])
test_scaled = scaler.transform(test[features_multi + [target_column]])

# Creazione sequenze
X_train, y_train = create_sequences_multi_output(train_scaled, sequence_length, n_future, target_index)
X_test, y_test = create_sequences_multi_output(test_scaled, sequence_length, n_future, target_index)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (13508, 168, 9), y_train shape: (13508, 336)
X_test shape: (2999, 168, 9), y_test shape: (2999, 336)


#### Caricamento del modello

In [ ]:
model_multi = load_model("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/modello_completo_multi.keras")
with open("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/history_multi.pkl", "rb") as file_pi:
    history_multi = pickle.load(file_pi)

y_pred_scaled = model_multi.predict(X_test)

y_pred = []
y_test_actual = []

for i in range(len(y_pred_scaled)):
    # Predizioni
    y_pred_reshaped = np.zeros((n_future, train_scaled.shape[1]))
    y_pred_reshaped[:, target_index] = y_pred_scaled[i]
    y_pred.append(scaler.inverse_transform(y_pred_reshaped)[:, target_index])

    # Valori reali
    y_test_reshaped = np.zeros((n_future, train_scaled.shape[1]))
    y_test_reshaped[:, target_index] = y_test[i]
    y_test_actual.append(scaler.inverse_transform(y_test_reshaped)[:, target_index])

y_pred = np.array(y_pred)
y_test_actual = np.array(y_test_actual)

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step


#### Modello

In [ ]:
tensorflow.keras.backend.clear_session()

In [ ]:
tensorflow.random.set_seed(42)
n_features_multi = X_train.shape[2]

model = Sequential()
model.add(Input(shape=(sequence_length_multi, n_features_multi)))  # Input

# LSTM + Dense
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(n_future))  # Output di 336 ore

# Compilazione
model.compile(optimizer='adam',loss='mse')

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 336)            │         5,712 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,264 (106.50 KB)

 Trainable params: 27,264 (106.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (13508, 168, 9)
y_train shape: (13508, 336)


In [ ]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train.reshape((y_train.shape[0], y_train.shape[1])),
    epochs=20, batch_size=32,validation_split=0.1,
    callbacks=[early_stop]
)

Epoch 1/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 33s 79ms/step - loss: 63896.6172 - val_loss: 0.1536
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - loss: 0.0550 - val_loss: 0.1256
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 22s 57ms/step - loss: 0.0453 - val_loss: 0.1093
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 39s 51ms/step - loss: 0.0328 - val_loss: 0.0297
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - loss: 0.0073 - val_loss: 0.0070
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 25s 66ms/step - loss: 0.0023 - val_loss: 0.0055
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 43s 70ms/step - loss: 0.0019 - val_loss: 0.0055
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 34s 51ms/step - loss: 0.0017 - val_loss: 0.0055
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - loss: 0.0017 - val_loss: 0.0057
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 21s 51ms/step - loss: 0.0016 - val_loss: 0.0059
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - loss: 0.0015 - val_loss: 0.0061
Epoch 12/20
380/380 ━━━━━━

In [ ]:
# Salva modello e history
model.save("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/modello_completo_multi.keras")
with open("/content/drive/MyDrive/Data Science/Data Science and Generative AI/Progetto finale/history_multi.pkl", "wb") as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
y_pred_scaled = model.predict(X_test)

y_pred = []
y_test_actual = []

for i in range(len(y_pred_scaled)):
    # Predizioni
    y_pred_reshaped = np.zeros((n_future, train_scaled.shape[1]))
    y_pred_reshaped[:, target_index] = y_pred_scaled[i]
    y_pred.append(scaler.inverse_transform(y_pred_reshaped)[:, target_index])

    # Valori reali
    y_test_reshaped = np.zeros((n_future, train_scaled.shape[1]))
    y_test_reshaped[:, target_index] = y_test[i]
    y_test_actual.append(scaler.inverse_transform(y_test_reshaped)[:, target_index])

y_pred = np.array(y_pred)
y_test_actual = np.array(y_test_actual)

94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step


In [ ]:
y_test_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

mse = mean_squared_error(y_test_flat, y_pred_flat)
mae = mean_absolute_error(y_test_flat, y_pred_flat)
r2 = r2_score(y_test_flat, y_pred_flat)

# Calcolo della loss (MSE) usando direttamente il modello, se serve
loss = model.evaluate(X_test, y_test, verbose=0)

print("------ Metriche Multi-step ------")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")
print(f"Loss da model.evaluate(): {loss:.4f}")

------ Metriche Multi-step ------
MSE:  4943765801.0575
MAE:  70196.6697
R²:   -423766506450.7045
Loss da model.evaluate(): 0.1279


#### Grafici

In [ ]:
#@title Loss - Test vs Validation
#Per modelli caricati usare i primi (history), per modelli addestrati sul momemnto (history.history)

loss = history_multi['loss']
val_loss = history_multi['val_loss']

fig_lstm = go.Figure()
fig_lstm.add_trace(go.Scatter(x=list(range(len(loss))), y=loss, mode='lines', name='Loss'))
fig_lstm.add_trace(go.Scatter(x=list(range(len(val_loss))), y=val_loss, mode='lines', name='Validation Loss'))
fig_lstm.update_layout(
    title= {'text': "Confronto Metriche di Performance: Loss - Test vs Validation",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Epochs",yaxis_title="Loss",showlegend=True,
    width=800,height=500,template='plotly_dark')

fig_lstm.show()

In [ ]:
print("y_pred[1].shape: ", y_pred[1].shape)
print("y_test_actual.shape: ", y_test_actual[:, 0].shape)

y_pred[1].shape:  (336,)
y_test_actual.shape:  (2999,)


In [ ]:
print(f"y_pred[:, 0] shape: {y_pred[:, 0].shape} | y_pred1[-1] shape: {y_pred[-1].shape} | y_pred1.mean(axis=1) shape: {y_pred.mean(axis=1).shape}")

y_pred[:, 0] shape: (2999,) | y_pred1[-1] shape: (336,) | y_pred1.mean(axis=1) shape: (2999,)


`y_pred.shape = (2999,336)`

- Le colonne `336` rappresentano le ore all'interno dell'orizzonte di previsione, che è di una settimana (168 ore). Ogni colonna è una previsione per una specifica ora futura all'interno di quella settimana.

- Le righe `2999` rappresentano le diverse finestre temporali di partenza per la previsione.



In [ ]:
df_y_pred = pd.DataFrame(y_pred)
df_y_pred

,0,1,2,3,4,5,6,7,8,9,...,326,327,328,329,330,331,332,333,334,335
0,72739.423292,74253.367846,71620.027449,73289.072891,73824.253069,73790.178463,74147.826670,74130.383912,73738.968263,73677.810080,...,75553.977520,74307.592339,75953.096815,75692.278645,75735.277357,76054.501531,76123.293736,75069.516082,74536.879213,75612.301614
1,72748.220438,74272.543003,71628.103785,73303.800328,73840.323832,73803.746873,74162.185512,74146.532490,73754.867015,73688.740984,...,75567.541834,74326.333372,75971.682219,75709.307756,75755.038170,76078.173552,76142.530326,75085.427121,74555.444139,75630.248119
2,72767.055667,74295.576126,71640.275628,73323.323603,73863.406101,73823.061277,74183.224124,74169.803152,73774.717930,73710.770708,...,75592.151727,74344.120152,76000.334375,75736.317615,75779.709495,76104.192299,76170.015263,75107.710766,74567.345678,75652.929028
3,72780.071184,74314.206580,71647.852313,73337.227844,73882.204471,73839.205759,74199.438230,74186.996084,73790.186654,73728.508342,...,75610.782181,74357.770472,76022.458296,75758.089322,75798.372714,76124.297135,76191.946695,75124.801310,74577.334616,75671.031162
4,72790.314043,74332.357860,71656.219431,73351.209900,73898.480010,73854.342748,74214.579314,74201.772668,73804.582356,73742.015319,...,75625.849546,74372.456954,76040.916739,75775.360069,75816.474848,76144.500264,76209.368975,75139.233872,74591.546020,75686.594083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,64985.185712,65278.624584,63502.813410,64986.475797,65878.644805,65896.550355,64629.704028,66420.840584,64809.726067,64418.871502,...,66539.696248,65770.007427,66538.119479,67326.971170,66088.518983,66430.706657,67122.261425,66432.791269,65603.467792,66359.317901
2995,64876.110115,65224.117502,63377.683431,64886.832148,65829.990200,65807.939998,64548.309952,66315.065964,64737.129137,64369.557520,...,66448.685926,65646.601656,66473.177048,67275.117976,65993.237048,66349.599266,67048.743006,66315.156065,65462.471825,66286.417903
2996,64776.138826,65190.902952,63270.889030,64808.567039,65790.927269,65745.921759,64486.254853,66233.549023,64680.430959,64330.044083,...,66376.813900,65553.346996,66425.816623,67231.513129,65932.852915,66299.441611,66995.284373,66228.314967,65363.139435,66220.594939
2997,64557.762379,65118.330595,62993.324338,64621.840658,65733.098731,65655.865689,64335.487010,66101.251910,64555.047059,64223.601896,...,66237.374225,65379.328968,66333.917576,67196.955254,65792.053533,66200.166558,66931.435443,66078.329365,65184.726930,66118.174538


In [ ]:
y_pred_step1 = y_pred[:, 0] # Estrai il primo valore previsto di ogni sequenza
print(y_pred_step1.shape)

y_pred_mean = y_pred.mean(axis=1) # Calcola la media lungo i 336 step
print(y_pred_mean.shape)

# Estrai i dati da y_test_actual
if y_test_actual.ndim == 2 and y_test_actual.shape[1] == 336: #controlla se i actual è di due dimensioni e ha la seconda dim di 336
    y_test_step1 = y_test_actual[:, 0]
    y_test_mean = y_test_actual.mean(axis=1)
else: #se risulta false, allora si tratta di un modelo single-step e non multi
    y_test_step1 = y_test_actual
    y_test_mean = y_test_actual

index = test_dates[sequence_length:]
actual = test['BTC_USDT_1h_close'].values[sequence_length:]

(2999,)
(2999,)


In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Predicted Step 1 vs Actual', 'Predicted Mean vs Actual'))

fig.add_trace(go.Scatter(x=index, y=y_test_step1, mode='lines', name='Actual', line=dict(color='white')), row=1, col=1)
fig.add_trace(go.Scatter(x=index, y=y_pred_step1, mode='lines', name='Predicted (Step 1)', line=dict(color='orange')), row=1, col=1)

fig.add_trace(go.Scatter(x=index, y=y_test_step1, mode='lines', name='Actual', line=dict(color='white'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=index, y=y_pred_mean, mode='lines', name='Predicted (Mean)', line=dict(color='salmon')), row=1, col=2)

fig.update_layout(
    title= {'text': "Confronto Previsioni vs Valori Reali",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}}
    ,showlegend=True, width=1500,height=500,template='plotly_dark')

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Value (step 1)", row=1, col=1)
fig.update_yaxes(title_text="Value mean", row=1, col=2)

fig.show()
print("\n------------------------------------------------------------------")
print("| Lo scarto medio tra la y_media e y_step1 è di:", round(np.abs(y_pred_mean - y_pred_step1).mean(), 2), "dollari |")
print("------------------------------------------------------------------")


------------------------------------------------------------------
| Lo scarto medio tra la y_media e y_step1 è di: 1249.65 dollari |
------------------------------------------------------------------


In [ ]:
scarto = np.abs(y_test_actual - y_pred)
media_scarto = np.mean(scarto)
print("Lo scarto medio tra la Y reale e la predetta è di:",round(media_scarto, 2), "dollari")

y_corretta = y_pred + media_scarto
print("\nLa nuova y_corretta:", y_corretta[:3], "\n\nLa y_pred:", y_pred[:3], "\n\nLa y_test:", y_test_actual[:3])

Lo scarto medio tra la Y reale e la predetta è di: 23768.01 dollari

La nuova y_corretta: [[96507.43651387 98021.38106813 95388.04067065 ... 98837.52930388
  98304.8924347  99380.31483562]
 [96516.23365963 98040.55622477 95396.11700745 ... 98853.44034265
  98323.45736103 99398.26134062]
 [96535.06888932 98063.58934756 95408.28884973 ... 98875.72398795
  98335.35889994 99420.94224995]] 

La y_pred: [[72739.42329183 74253.36784608 71620.02744861 ... 75069.51608183
  74536.87921266 75612.30161358]
 [72748.22043759 74272.54300272 71628.10378541 ... 75085.42712061
  74555.44413898 75630.24811857]
 [72767.05566727 74295.57612552 71640.27562768 ... 75107.7107659
  74567.34567789 75652.9290279 ]] 

La y_test: [[91045.12 91696.5  91815.01 ... 94467.81 95121.63 95842.52]
 [91696.5  91815.01 91956.47 ... 95121.63 95842.52 96421.28]
 [91815.01 91956.47 91937.45 ... 95842.52 96421.28 96855.08]]


In [ ]:
print("y_corretta.shape:", y_corretta.shape)
y_corretta_step1 = y_corretta[:, 0]

y_corretta.shape: (2999, 336)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=index, y=y_test_step1, mode='lines', name='Actual - Step 1', line=dict(color='white')))
fig.add_trace(go.Scatter(x=index, y=y_corretta_step1, mode='lines', name='Forecast Corretta', line=dict(color='deepskyblue')))
fig.update_layout(
    title= {'text': "Previsioni vs Valori Reali - Primo Step (1h) [y_corretta]",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Epochs",yaxis_title="Loss",showlegend=True,
    width=1200,height=700,template='plotly_dark')

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price BTC")

fig.show()

Esempio:

Se `y_pred.shape = (3000, 336)`, allora:
- `y_pred[0]` = la previsione dei prossimi 336 step a partire dal primo input di test.
- `y_pred[2999]` (cioè `y_pred[-1]`) = previsione a 336 step partendo dall’ultima finestra disponibile.

Spesso si usa `y_pred[-1]` quando vuoi visualizzare solo l’ultima previsione, quella più “futura”.

In [ ]:
#@title Previsione per le prossime 2
#future_forecast = y_pred[-1]  # Ultima previsione: 336 step -> per effettuare la previsione delle due future settimane
future_forecast = y_corretta[-1]
#y_pred[-1] restituisce l’ultima previsione fatta dal modello, cioè l’output relativo all’ultima finestra del tuo set di test.

last_date = test_dates.iloc[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(hours=1), periods=336, freq='h')

# Fit polinomiale
y = np.array(future_forecast)
x = np.arange(len(y))
degree = 3
coeffs = np.polyfit(x, y, deg=degree)
poly_model = np.poly1d(coeffs)
y_fit = poly_model(x)

# Residui, stderr e intervallo di confidenza
residuals = y - y_fit
stderr = np.sqrt(np.sum(residuals**2) / (len(x) - degree - 1))
alpha = 0.10  # 90% CI
t_value = t.ppf(1 - alpha/2, df=len(x) - degree - 1)
conf_interval = t_value * stderr
y_upper = y_fit + conf_interval
y_lower = y_fit - conf_interval

# Media mobile
rolling_avg = pd.Series(y).rolling(window=24).mean()

fig = go.Figure()
# Confidence interval
fig.add_trace(go.Scatter(x=future_dates, y=y_upper, mode='lines', line=dict(width=0), showlegend=False))
fig.add_trace(go.Scatter(x=future_dates, y=y_lower, mode='lines', line=dict(width=0), fill='tonexty', fillcolor='rgba(255, 255, 0, 0.10)', name='Conf. Interval', showlegend=True ))
# Forecast
fig.add_trace(go.Scatter(x=future_dates, y=y, mode='lines', name='Forecast', line=dict(color='deepskyblue')))
# Rolling average
fig.add_trace(go.Scatter(x=future_dates, y=rolling_avg, mode='lines', name='Media Mobile (24h)',line=dict(color='salmon')))
# Polynomial trend
fig.add_trace(go.Scatter(x=future_dates, y=y_fit, mode='lines', name=f'Polynomial Trend (deg {degree})', line=dict(color='yellow')))

fig.update_layout(
    title= {'text': "Previsione per le prossime 2 settimane con Curva Polinomiale e Intervallo di Confidenza",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Date",yaxis_title="Prezzo previsto",showlegend=True, width=1300,height=700,template='plotly_dark', legend=dict(x=0, y=1))

fig.show()

In [ ]:
def plot_r2_sub_comparison(y_actual, y_pred1, y_pred2, label1='Predetta', label2='Corretta'):
    r2_1 = r2_score(y_actual, y_pred1)
    r2_2 = r2_score(y_actual, y_pred2)

    min_val = min(y_actual.min(), y_pred1.min(), y_pred2.min())
    max_val = max(y_actual.max(), y_pred1.max(), y_pred2.max())

    fig = make_subplots(rows=1, cols=2, subplot_titles=[f'{label1} Prediction - R² = {r2_1:.3f}',f'{label2} Prediction - R² = {r2_2:.3f}'])
    # Subplot 1: predetta
    fig.add_trace(go.Scatter(x=y_actual, y=y_pred1, mode='markers',marker=dict(color='lightsalmon', opacity=0.6),name=label1), row=1, col=1)
    fig.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val],mode='lines', line=dict(color='white', dash='dash'),name='Ideal', showlegend=False), row=1, col=1)

    # Subplot 2: corretta
    fig.add_trace(go.Scatter(x=y_actual, y=y_pred2, mode='markers',marker=dict(color='deepskyblue', opacity=0.6),name=label2), row=1, col=2)
    fig.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val],mode='lines', line=dict(color='white', dash='dash'),name='Ideal', showlegend=False), row=1, col=2)

    # Layout generale
    fig.update_layout(title={'text': 'Confronto Scatter Plot R² tra modelli','x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
        width=1200,height=600,legend=dict(x=0, y=1),template='plotly_dark')

    # Titoli assi
    for col in [1, 2]:
        fig.update_xaxes(title_text='Actual Values', row=1, col=col)
        fig.update_yaxes(title_text='Predicted Values', row=1, col=col)

    fig.show()

In [ ]:
plot_r2_sub_comparison(y_test_step1, y_pred_step1, y_corretta_step1)

### 2.5. Classification

L’obiettivo di questo modello di classificazione è analizzare e prevedere se il prezzo del Bitcoin sarà in rialzo o in ribasso.

In conclusione, dalle analisi del modello applicato per una previsione settimanale è emerso che il sistema fatica a determinare correttamente se la chiusura del prezzo di BTC sarà superiore o inferiore rispetto all’apertura. Il modello ha raggiunto un’accuratezza di 0.49, il che corrisponde a una performance simile a quella di un modello casuale.

#### Previsioni a un giorno

In [ ]:
df_class_with_dt = pd.DataFrame()
df_class_with_dt['Datetime'] = df_lstm_with_dt['Datetime']
df_class_with_dt['BTC +1g'] = df_lstm_with_dt['BTC_USDT_1h_close'].pct_change(giorno).shift(-giorno)
df_class_with_dt['BTC -1s'] = df_lstm_with_dt['BTC_USDT_1h_close'].pct_change(settimana)
df_class_with_dt.dropna(inplace=True)

df_class = df_class_with_dt.drop(columns=['Datetime'])
df_class_with_dt.head()

,Datetime,BTC +1g,BTC -1s
168,2023-04-23 00:00:00,0.009448,-0.090418
169,2023-04-23 01:00:00,0.007240,-0.088796
170,2023-04-23 02:00:00,0.005534,-0.088430
171,2023-04-23 03:00:00,0.008357,-0.088932
172,2023-04-23 04:00:00,0.003478,-0.088964


In [ ]:
target_class = 'BTC +1g'
df_class[target_class] = df_class[target_class].apply(lambda x: 1 if x > 0 else 0)
features_class = [f for f in df_class.columns if f != target_class]

df_class.head()

,BTC +1g,BTC -1s
168,1,-0.090418
169,1,-0.088796
170,1,-0.088430
171,1,-0.088932
172,1,-0.088964


In [ ]:
train_class, test_class = train_test_split(df_class, test_size=0.2, shuffle=False, random_state=42)

model_lgbmc = LGBMClassifier(verbose=0)
model_lgbmc.fit(train_class[features_class], train_class[target_class])
preds = pd.Series(model_lgbmc.predict(test_class[features_class]))

accuracy = accuracy_score(test_class[target_class], preds)
balanced_accuracy = balanced_accuracy_score(test_class[target_class], preds)
average_precision = average_precision_score(test_class[target_class], preds)

print("Precisione Media:", round(average_precision,2))
print("Accuratezza:", round(accuracy,2))
print("Punteggio di Accuratezza Bilanciato:", round(balanced_accuracy,2))
print("\n-------------------------------------------------------")
print("\n",classification_report(test_clas[target], preds))
print("-------------------------------------------------------")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Precisione Media: 0.5
Accuratezza: 0.52
Punteggio di Accuratezza Bilanciato: 0.52

-------------------------------------------------------

               precision    recall  f1-score   support

           0       0.54      0.38      0.45      1781
           1       0.50      0.66      0.57      1684

    accuracy                           0.52      3465
   macro avg       0.52      0.52      0.51      3465
weighted avg       0.52      0.52      0.51      3465

-------------------------------------------------------


#### Previsioni a una settimana

In [ ]:
target_class = 'BTC +1s'

df_class_with_dt = df_lstm_with_dt.copy()
df_class_with_dt[target_class] = df_lstm_with_dt['BTC_USDT_1h_close'].pct_change(settimana).shift(-settimana)
df_class_with_dt['BTC -1s'] = df_lstm_with_dt['BTC_USDT_1h_close'].pct_change(settimana)
df_class_with_dt.dropna(inplace=True)

df_class_with_dt = df_class_with_dt.drop(columns=['BTC_USDT_1h_close'])
df_class = df_class_with_dt.drop(columns=['Datetime'])
df_class_with_dt.head()

,Datetime,BNB_USDT_1h_close,DOGE_USDT_1h_close,ETH_USDT_1h_close,SOL_USDT_1h_close,XRP_USDT_1h_close,cattle_Close LE=F,corn_Close ZC=F,crude_Close CL=F,gold_Close GC=F,...,Russell_Close ^RUT,S&P_Close ^GSPC,S&P_Close ^GSPTSE,VIX_Close ^VIX,funding_rate,fear_gread_index,google_trends_buy_crypto,google_trends_bitcoin,BTC +1s,BTC -1s
168,2023-04-23 00:00:00,330.6138,0.07964,1856.90,21.58,0.4654,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,0.058105,-0.090418
169,2023-04-23 01:00:00,329.8215,0.07947,1855.40,21.60,0.4656,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,0.056052,-0.088796
170,2023-04-23 02:00:00,329.8033,0.07980,1859.34,21.66,0.4653,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,0.057512,-0.088430
171,2023-04-23 03:00:00,330.3639,0.08036,1861.94,21.72,0.4671,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,0.058840,-0.088932
172,2023-04-23 04:00:00,331.2622,0.08028,1864.36,21.88,0.4692,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,0.058130,-0.088964


##### Modello semplice

In [ ]:
target_class = 'BTC +1s'
df_class_with_dt[target_class] = df_class_with_dt[target_class].apply(lambda x: 1 if x > 0 else 0)
features_class = [col for col in df_class_with_dt.columns if col not in ['Datetime', target_class]]

df_class_with_dt.head()

,Datetime,BNB_USDT_1h_close,DOGE_USDT_1h_close,ETH_USDT_1h_close,SOL_USDT_1h_close,XRP_USDT_1h_close,cattle_Close LE=F,corn_Close ZC=F,crude_Close CL=F,gold_Close GC=F,...,Russell_Close ^RUT,S&P_Close ^GSPC,S&P_Close ^GSPTSE,VIX_Close ^VIX,funding_rate,fear_gread_index,google_trends_buy_crypto,google_trends_bitcoin,BTC +1s,BTC -1s
168,2023-04-23 00:00:00,330.6138,0.07964,1856.90,21.58,0.4654,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,1,-0.090418
169,2023-04-23 01:00:00,329.8215,0.07947,1855.40,21.60,0.4656,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,1,-0.088796
170,2023-04-23 02:00:00,329.8033,0.07980,1859.34,21.66,0.4653,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,1,-0.088430
171,2023-04-23 03:00:00,330.3639,0.08036,1861.94,21.72,0.4671,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,1,-0.088932
172,2023-04-23 04:00:00,331.2622,0.08028,1864.36,21.88,0.4692,164.399994,614.5,77.949997,1994.099976,...,1790.097534,4131.64502,20686.290039,16.67,0.0001,56.0,16,31,1,-0.088964


In [ ]:
train_class, test_class = train_test_split(df_class_with_dt, test_size=0.2, shuffle=False, random_state=42)

model_lgbmc = LGBMClassifier(verbose=0)
model_lgbmc.fit(train_class[features_class], train_class[target_class])
preds = pd.Series(model_lgbmc.predict(test_class[features_class]))

accuracy = accuracy_score(test_class[target_class], preds)
balanced_accuracy = balanced_accuracy_score(test_class[target_class], preds)
average_precision = average_precision_score(test_class[target_class], preds)

print("Precisione Media:", round(average_precision,2))
print("Accuratezza:", round(accuracy,2))
print("Punteggio di Accuratezza Bilanciato:", round(balanced_accuracy,2))
print("\n-------------------------------------------------------")
print("\n",classification_report(test_class[target_class], preds))
print("-------------------------------------------------------")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Precisione Media: 0.51
Accuratezza: 0.59
Punteggio di Accuratezza Bilanciato: 0.6

-------------------------------------------------------

               precision    recall  f1-score   support

           0       0.67      0.47      0.55      1872
           1       0.53      0.73      0.62      1564

    accuracy                           0.59      3436
   macro avg       0.60      0.60      0.58      3436
weighted avg       0.61      0.59      0.58      3436

-------------------------------------------------------


##### Modello con Purging

Obiettivo:

Addestrare e valutare un modello di classificazione per prevedere se il prezzo di BTC salirà o scenderà nel futuro (1 se sale, 0 se scende), usando dati storici in stile time-series. Il modello viene riaddestrato ogni mese simulando uno scenario realistico di finanza.

- settimana: `168`
- Target: `pct_change(168).shift(-168)`
- Frequenza split: `n_splits = len(df) // 168`
- Gap temporale: `gap = 168`
- Modello: `LightGBM`
- Valutazione: **Predizione se BTC salirà tra 1 settimana**


In [ ]:
settimana = 168

**Problema della “contaminazione” dei dati nelle serie temporali**

Nel caso delle serie temporali (dove i dati sono orari) i valori a tempo t sono spesso correlati con quelli a tempo t-1, t-2, …, a causa delle dinamiche di mercato. In altre parole, se il tuo modello impara da dati che sono troppo vicini nel tempo tra il set di addestramento e il set di test, può “contaminare” il test con informazioni che appartengono al periodo successivo.

Questa contaminazione avviene perché il modello potrebbe imparare pattern che sono legati al passato immediato (es. un cambiamento di prezzo che potrebbe essere prevedibile con una finestra temporale molto stretta). In altre parole, potrebbe guardare nel futuro durante l’addestramento, violando la regola fondamentale delle serie temporali: i modelli devono fare previsioni sul futuro senza conoscere il futuro stesso.

**Perché aggiungere un gap?**

L’introduzione di un gap (o “purging”) tra il set di addestramento e quello di test serve proprio a evitare che l’algoritmo faccia previsioni su dati che sono troppo vicini nel tempo e quindi potenzialmente “sintetici”. Questo gap è una tecnica usata per garantire che i dati di test siano abbastanza distanti dai dati di addestramento, evitando che il modello si adatti a pattern che potrebbero non essere generalizzabili.

In sintesi, l’introduzione del gap riduce il rischio di data leakage (perdita di dati) tra addestramento e test, migliorando la capacità del modello di generalizzare a nuovi dati.


In [ ]:
# Parametri
df_class_with_dt[target_class] = df_class_with_dt[target_class].apply(lambda x: 1 if x > 0 else 0)
features_class = [col for col in df_class_with_dt.columns if col not in ['Datetime', target_class]]

#replace degli spazi con _, per evitare gli warnings da LGMBC
df_class_with_dt.columns = df_class_with_dt.columns.str.replace(' ', '_')
target_class = target_class.replace(' ', '_')
features_class = [f.replace(' ', '_') for f in features_class]

# Parametri TimeSeriesSplit
min_train_size = int(len(df_class_with_dt) * 0.5) # almeno 50% per il primo training
n_splits = len(df_class_with_dt) // settimana # retraining settimanale: 168 ore
tscv = TimeSeriesSplit(n_splits=n_splits, gap=settimana)

`gap=168`: significa che, quando creiamo uno split, il set di addestramento e il set di test non saranno immediatamente consecutivi. Verranno separati da un intervallo di 168 unità di tempo. Questo gap assicura che i dati nel set di addestramento non siano troppo vicini ai dati nel set di test.

Questo significa che tra il momento in cui l’algoritmo termina di usare i dati di addestramento e inizia a testare il modello sui dati successivi, ci sarà uno “stacco” di 168 unità di tempo (in questo caso, probabilmente 168 ore, ovvero una settimana). Questo crea una separazione tra il set di addestramento e il set di test, impedendo che i dati del set di addestramento influenzino troppo il set di test, riducendo così il rischio di contaminazione.


In [ ]:
for i, (train_index, test_index) in enumerate(tscv.split(df_class_with_dt)):
    train_dates = df_class_with_dt.iloc[train_index]['Datetime']
    test_dates = df_class_with_dt.iloc[test_index]['Datetime']

    print(f"🔁 Split {i+1}")
    print(f"  🟩 Train: {train_dates.min()} → {train_dates.max()}")
    print(f"  🟥 Test:  {test_dates.min()} → {test_dates.max()}")
    print("-" * 54)

🔁 Split 1
  🟩 Train: 2023-04-23 00:00:00 → 2023-04-26 06:00:00
  🟥 Test:  2023-05-03 07:00:00 → 2023-05-10 04:00:00
------------------------------------------------------
🔁 Split 2
  🟩 Train: 2023-04-23 00:00:00 → 2023-05-03 04:00:00
  🟥 Test:  2023-05-10 05:00:00 → 2023-05-17 02:00:00
------------------------------------------------------
🔁 Split 3
  🟩 Train: 2023-04-23 00:00:00 → 2023-05-10 02:00:00
  🟥 Test:  2023-05-17 03:00:00 → 2023-05-24 00:00:00
------------------------------------------------------
🔁 Split 4
  🟩 Train: 2023-04-23 00:00:00 → 2023-05-17 00:00:00
  🟥 Test:  2023-05-24 01:00:00 → 2023-05-30 22:00:00
------------------------------------------------------
🔁 Split 5
  🟩 Train: 2023-04-23 00:00:00 → 2023-05-23 22:00:00
  🟥 Test:  2023-05-30 23:00:00 → 2023-06-06 20:00:00
------------------------------------------------------
🔁 Split 6
  🟩 Train: 2023-04-23 00:00:00 → 2023-05-30 20:00:00
  🟥 Test:  2023-06-06 21:00:00 → 2023-06-13 18:00:00
-----------------------------

In [ ]:
#Addestramento modello con retraining settimanale
all_preds = []
all_true = []
all_dates = []

for train_index, test_index in tscv.split(df_class_with_dt):
    if len(train_index) < min_train_size:
        continue

    train_split = df_class_with_dt.iloc[train_index]
    test_split = df_class_with_dt.iloc[test_index]

    model = LGBMClassifier(n_estimators=100,learning_rate=0.05,max_depth=5,num_leaves=31,subsample=0.8,colsample_bytree=0.8,random_state=42,verbose=-1)
    model.fit(train_split[features_class], train_split[target_class])

    preds = model.predict(test_split[features_class])
    all_preds.extend(preds)
    all_true.extend(test_split[target_class].values)
    all_dates.extend(df_class_with_dt.iloc[test_index]['Datetime'].values)

In [ ]:
# 📊 Metriche
all_preds = pd.Series(all_preds, name="Predictions")
all_true = pd.Series(all_true, name="True Values")
all_dates = pd.to_datetime(all_dates)
print("-" * 32)
print("📈 Average Precision Score:", round(average_precision_score(all_true, all_preds),2))
print("🎯 Accuracy Score:", round(accuracy_score(all_true, all_preds),2))
print("⚖️ Balanced Accuracy Score:", round(balanced_accuracy_score(all_true, all_preds),2))
print("-" * 32)

--------------------------------
📈 Average Precision Score: 0.52
🎯 Accuracy Score: 0.49
⚖️ Balanced Accuracy Score: 0.48
--------------------------------


In [ ]:
plot_df_class = pd.DataFrame({
    'Date': dates_all,
    'True': all_true,
    'Predicted': all_preds
})
plot_df_class.dropna(inplace=True)

In [ ]:
hours_focus = 24 * 7 * 4 #672 ore, ovvere le ultime 4 sett
n_sett = hours_focus//(settimana)
last_hours = plot_df_class.tail(hours_focus)

fig = go.Figure()
fig.add_trace(go.Scatter(x=last_hours['Date'], y=last_hours['True'], mode='lines', name='True', line=dict(color='white', width=1)))
fig.add_trace(go.Scatter(x=last_hours['Date'], y=last_hours['Predicted'], mode='lines', name='Predicted', line=dict(color='deepskyblue', width=1)))

fig.update_layout(
    title= {'text': f"Classificazione: Previsioni vs Valori Reali, delle ultime {n_sett} settimane",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Date",yaxis_title="Class (0 = Down, 1 = Up)",yaxis=dict(tickmode='array',tickvals=[0, 1],ticktext=['0', '1']), showlegend=True,
    width=1200,height=400,template='plotly_dark')

fig.show()

In [ ]:
cm = confusion_matrix(all_true, all_preds); labels = ["Down (0)", "Up (1)"]

fig_cm = go.Figure(data=go.Heatmap(z=cm, x=labels, y=labels, colorscale='Blues', text=cm, texttemplate="%{text}",
                                   hovertemplate="Predicted: %{x}<br>Actual: %{y}<br>Count: %{z}<extra></extra>"))
fig_cm.update_layout(
    title= {'text': f"Confusion Matrix",'x': 0.5,'xanchor': 'center','font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    xaxis_title="Predicted Label", yaxis_title="True Label", width=600, height=600, template='plotly_dark')

fig_cm.show()

## Concusioni

- I modelli LSTM mostrano buona capacità predittiva a breve termine, ma scarsa generalizzazione a lungo termine senza l’integrazione di dati esterni.
- I modelli di classificazione non hanno superato le performance di un modello casuale, confermando l’insufficienza dei soli dati storici per la previsione del market trend.

###🔍 Analisi Futura
Per migliorare l'approccio predittivo, si consiglia di:
**integrare fattori esogeni** come cambiamenti nei tassi d’interesse, politiche monetarie o eventi geopolitici. Ad esempio, eventi economici globali, politiche monetarie, e notizie legate a Bitcoin.
Ciò permetterebbe la costruzione di modelli scenario-based, in grado di simulare l’impatto di specifici eventi futuri sul sentiment di mercato.
